In [ ]:
import functions as fc
import pandas as pd

In [ ]:
fc.download_tgz_raw()

In [ ]:
fc.import_tgz("2018", "./data2018")

In [ ]:
fc.import_tgz("2019", "./data2019")

In [ ]:
fc.import_tgz("2020", "./data2020")

In [ ]:
df_final = fc.transform_rcs("2018")

In [ ]:
df_final.head()

In [ ]:
fc.export_2_minio("2018", df_final)

In [ ]:
fc.export_2_minio("2019", df_final)

In [ ]:
fc.export_2_minio("2020", df_final)

In [ ]:
from urllib.request import urlopen
import re
from bs4 import BeautifulSoup
import requests
from zipfile import ZipFile

def isDirectory(url):
    if(url.endswith('/') and ".." not in url):
        return True
    else:
        return False
list_urls = []
def findLinks(url):
    page = requests.get(url).content
    bsObj = BeautifulSoup(page, 'html.parser')
    maybe_directories = bsObj.findAll('a', href=True)
    list_df = []
    
    for link in maybe_directories:
        #print(link['href'])
        # print(isDirectory(link['href']))
        if(isDirectory(link['href'])):
            newUrl = url + link['href']   
            print(newUrl)   
            list_urls.append(newUrl)  
            print(list_urls) 
            findLinks(newUrl) #recursion happening here$
            
            
        else:
            if(link['href'].endswith('.zip')):
                print("GOTCHA!") #now safe and download
                target_path = link['href']
                response = requests.get(list_urls[-1] + link['href'], stream=True)
                if response.status_code == 200:
                    with open(target_path, 'wb') as f:
                        f.write(response.raw.read())
                
                # zip_file = ZipFile(list_urls[-1] + link['href'])         
                # temp_df = [open_complex_file(zip_file, text_file.filename)
                # for text_file in zip_file.infolist()
                # if text_file.filename.endswith('.csv')]
                # list_df.append(temp_df)

startUrl = "http://data.cquest.org/inpi_rncs/imr/stock/2017/"
findLinks(startUrl)

In [ ]:
zip_file = ZipFile(list_urls[-1] + link['href'])         
 temp_df = [open_complex_file(zip_file, text_file.filename)
for text_file in zip_file.infolist()
if text_file.filename.endswith('.csv')]
 list_df.append(temp_df)